In [2]:
import pandas as pd

df=pd.read_table(r'D:/Projects/Python/All-Machine-Learning\Projects\Watsapp Group Analysis\WhatsApp Chat.txt',error_bad_lines=False)
df.columns=["Raw"]
df_new=df['Raw'].str.split(': ', 1, expand=True)
df_new.columns=['Time','Message']
#df_new
MsgTable=df_new['Time'].str.split('- ',1,expand=True)
MsgTable.columns=['Timestamp','Name']
MsgTable['Message']=df_new['Message']
MsgTable.iloc[1000]

b'Skipping line 2029: expected 1 fields, saw 2\nSkipping line 4430: expected 1 fields, saw 2\nSkipping line 5962: expected 1 fields, saw 2\nSkipping line 5963: expected 1 fields, saw 2\nSkipping line 5964: expected 1 fields, saw 2\nSkipping line 5965: expected 1 fields, saw 2\nSkipping line 5966: expected 1 fields, saw 2\nSkipping line 5967: expected 1 fields, saw 2\nSkipping line 5968: expected 1 fields, saw 2\nSkipping line 5969: expected 1 fields, saw 2\nSkipping line 5970: expected 1 fields, saw 2\nSkipping line 5971: expected 1 fields, saw 2\nSkipping line 5972: expected 1 fields, saw 2\nSkipping line 5973: expected 1 fields, saw 2\nSkipping line 5974: expected 1 fields, saw 2\nSkipping line 46025: expected 1 fields, saw 2\nSkipping line 46026: expected 1 fields, saw 2\nSkipping line 46027: expected 1 fields, saw 2\nSkipping line 46029: expected 1 fields, saw 2\nSkipping line 46030: expected 1 fields, saw 2\n'


Timestamp                     27/12/16, 4:38 PM 
Name                             Arun Bhadardwaj
Message      Lol 🙏🏼🙏🏼 Beda Kesha Maryade hogatte
Name: 1000, dtype: object

In [ ]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

MsgTable['Message']=MsgTable['Message'].astype(str)
MsgList = MsgTable['Message'].tolist()
print(MsgList[200:202])

stop_words = set(stopwords.words('english'))
WordList=' '.join(MsgTable['Message']).lower().split()
filtered_WordList = [w for w in WordList if not w in stop_words]


from collections import Counter
df = pd.DataFrame.from_dict(dict(Counter(filtered_WordList)), orient='index').reset_index()
df.columns =['Word','Count']
DropWords = ['','and','-','u','to','for','with','of','in','<omitted','image','<media','omitted>','it','none','i','is','will','the','you']
df['Word'].replace(DropWords, np.nan, inplace=True)
df.dropna(subset=['Word'], inplace=True)
df.sort(['Count'],ascending=False,inplace=True)
df.head()
df.plot(x="Word",y="Count",kind="bar")
plt.show()


['A man threw a pot of boiling water above his head in -25 degree centigrade resulting in a lovely snow cloud', '<Media omitted>']


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
